#  Imports verses from Tanakh into a LaTeX-friendly format

In [ ]:
#this defines which book is where and what the exact names are
#to look up the names properly

In [5]:
torah = 'torah.csv'
megillot = 'chamesh megillot.csv'
navi1 = 'neviim rishonim.csv'
navi2 = 'neviim achronim.csv'
emet = 'sifrei emet.csv'
ketuvim = 'ketuvim.csv'

In [23]:
sefarim = {
    "ספר בראשית":torah,
    "ספר שמות":torah,
    "ספר ויקרא":torah,
    "ספר במדבר":torah,
    "ספר דברים":torah,
    "ספר יהושע":navi1,
    "ספר שופטים":navi1,
    "ספר שמואל":navi1,
    "ספר מלכים":navi1,
    "ספר ישעיהו":navi2,
    "ספר ירמיהו":navi2,
    "ספר יחזקאל":navi2,
    "ספר תרי עשר":navi2,
    "ספר תהלים":emet,
    "ספר משלי":emet,
    "ספר איוב":emet,
    "מגילת שיר השירים":megillot,
    "מגילת רות":megillot,
    "מגילת איכה":megillot,
    "מגילת קהלת":megillot,
    "מגילת אסתר":megillot,
    "ספר דניאל":ketuvim
    #needs support for  aleph/bet, trei-asar, ezra-nehemiah, divrei-hayamim
}

In [24]:
sefer_name = {
    "gen":"ספר בראשית",
    "ex":"ספר שמות",
    "lev":"ספר ויקרא",
    "num":"ספר במדבר",
    "dev":"ספר דברים",
    "josh":"ספר יהושע",
    "judges":"ספר שופטים",
    "sam":"ספר שמואל",
    "kings":"ספר מלכים",
    "is":"ספר ישעיהו",
    "jer":"ספר ירמיהו",
    "ezek":"ספר יחזקאל",
    #"ספר תרי עשר",
    "psalms":"ספר תהלים",
    "prov":"ספר משלי",
    "job":"ספר איוב",
    "song":"מגילת שיר השירים",
    "ruth":"מגילת רות",
    "lam":"מגילת איכה",
    "ecc":"מגילת קהלת",
    "esther":"מגילת אסתר",
    "dan":"ספר דניאל"
    #needs support for  aleph/bet, trei-asar, ezra-nehemiah, divrei-hayamim
}

In [25]:
#this parses the formatting notes in the 3rd column of the spreadsheet

In [26]:
format = {
    "//{{פפ}}//":"\p",
    "{{סס}}":"\s",
    "{{פפ}}":"\p",
    "{{פפפ}}":"\p",
    "{{רווח בסוף שורה}}":"\p",
    "{{ססס}}":"\s",
    "{{סס2}}":"\s",
    "//{{סס}}":"\s",
    "{{פפ}}//":"\p",
    "{{מ:אין פרשה בתחילת פרק}}":"",
    "{{ר4}}":"\hfill"
    #{{פסקא באמצע פסוק|משתנה|הערות}}
}

In [27]:
def format_parser(format_in):
    blank = '__'
    new_book = 'ספר חדש'
    
    if format_in == blank:
        return ''
    elif format_in in format:
        return format[format_in]
    elif new_book in format_in:
        index = format_in.find('|')
        book_name_start = index+1
        book_name_end = format_in.find('}',index)
        book_name = format_in[book_name_start:book_name_end]
        return '\\newbook{'+book_name+'}'
    else:
        return format_in

In [28]:
#this looks for a note about the aliya in the 4th column of the spreadsheet.
#maybe functionality for sedarim will be added over time.

In [29]:
#this would change aliyot info to exact commands
#the current plan is to just have the aliya text be the argument of the \aliya{} command, so this is not needed.
#if used, this needs support for double parshiyot.
aliya_commands = {
    "[לוי]":"\\aliya{levi}",
    "[ישראל]":"\\aliya{yisrael}",
    "[ע\"כ ישראל]":"\\aliya{end_yisrael}",
    "שני":"\\aliya{sheni}",
    "שלישי":"\\aliya{shelishi}",
    "רביעי":"\\aliya{revii}",
    "חמישי":"\\aliya{chamishi}",
    "ששי":"\\aliya{shishi}",
    "שביעי":"\\aliya{shevii}",
    "מפטיר":"\\aliya{maftir}"
}

In [30]:
def aliya_parser(aliya_in):
    aliya = 'עלייה'
    aliya_command = ''
    seder_command = ''
    output = ''
    if aliya in aliya_in:
        aliya_start=aliya_in.find(aliya)
        while aliya_start<len(aliya_in) and aliya_in[aliya_start]!='=':
            aliya_start=aliya_start+1
        aliya_end = aliya_start+1
        while aliya_end<len(aliya_in) and aliya_in[aliya_end]!='}':
            aliya_end = aliya_end+1
        aliya_number = aliya_in[aliya_start+1:aliya_end]
        if '\'' in aliya_number:
            parsha_name = aliya_number.replace('\'','')
            aliya_command = '\\parsha{'+parsha_name+'}'
        else:
            aliya_command = '\\aliya{'+aliya_number+'}'
    
    seder = 'סדר'
    if seder in aliya_in:
        seder_start=aliya_in.find(seder)
        while seder_start<len(aliya_in) and aliya_in[seder_start]!='=':
            seder_start=seder_start+1
        seder_end = seder_start+1
        while seder_end<len(aliya_in) and aliya_in[seder_end]!='|':
            seder_end = seder_end+1
        seder_command = '\seder{'+aliya_in[seder_start+1:seder_end]+'}'
    
    output = seder_command + aliya_command
    return output

In [31]:
#this parses notes embedded in the tanakh-text.

In [32]:
text_format = {
    "{{מ:לגרמיה}}":" ׀ ",
    "{{מ:פסק}}":" ׀ "
    
}

In [37]:
def nusach_parser(note):
    main_nusach_start = note.find('נוסח')+5
    main_nusach_end = note.find('=',main_nusach_start)-1
    main_nusach = note[main_nusach_start:main_nusach_end]
    secondary_nusach = note[main_nusach_end+2:len(note)-4]
    output = main_nusach+'\\footnote{'+secondary_nusach+'}'
    return output

In [38]:
def text_parser(pasuk):
    #this loop replaces the items which can be replaced by a specific character(s) using the text_format dict
    for entry in text_format:
        if entry in pasuk:
            pasuk = pasuk.replace(entry,text_format[entry])
    
    #this parses the nusach-notes and turns them into footnotes, using the nusach_parser
    nusach = 'נוסח'
    while nusach in pasuk:
        index = pasuk.find(nusach)
        note_start = index-2
        extra_brace = 0
        while index<len(pasuk) and pasuk[index]!='}':
            if pasuk[index]=='{':
                extra_brace = extra_brace+1
            index = index+1
        while index<len(pasuk) and extra_brace != 0:    
            if pasuk[index]=='}':
                extra_brace = extra_brace-1
            index = index+1
        note_end = pasuk.find('}}',index)+2
        nusach_note = pasuk[note_start:note_end]
        nusach_new = nusach_parser(nusach_note)
        pasuk = pasuk.replace(nusach_note,nusach_new)
    
    #needs kri ukhtiv
    kq = 'כו"ק'
    qk = 'קו"כ'
    qk_em='קו"כ-אם'
    
    
    #needs special characters
    #needs words with alternate kamatz katan forms
    return pasuk

In [39]:
#this pulls verses from the Torah and runs them through the above parsers

In [42]:
import csv

book_in = 'gen'
book_in = sefer_name[book_in]
chelek = sefarim[book_in]

perek_start = 'א'
perek_end = 'י'
pasuk_start = 'א'
pasuk_end = 'א'
output = ''
output_file = 'output.txt'

with open(chelek, 'r', encoding='utf-8') as csvDataFile, open(output_file,'w',encoding='utf-8') as outfile:
    csvReader = csv.reader(csvDataFile)
    in_range = False
    for row in csvReader:
        if row[1]=='0' or row[1]=='תתת':
            continue
        book_perek = row[0].split('/')
        pasuk = row[1]
        if book_in == book_perek[0] and perek_start==book_perek[1] and pasuk_start==pasuk:
            in_range = True
        
        if in_range==True:
            if pasuk == 'א':
                perek_command = '\\perek{'+book_perek[1]+'}'
            else:
                perek_command = ''
            par = format_parser(row[2])+perek_command+'\n'
            aliya = aliya_parser(row[3])+'\n'
            text = text_parser(row[4])
            line = par+aliya+text
            print(line)
            outfile.write(line)        
        if perek_end==book_perek[1] and pasuk_end==pasuk:
            break

\newbook{בראשית}\perek{א}
\seder{א}\parsha{בראשית}
{{מ:אות-ג|בְּ}}רֵאשִׁ֖ית\footnote{מ"ס-ל ומ"ש|ל=בְּרֵאשִׁ֖ית (באות רגיל} בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃


וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֙הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְה֑וֹם וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃


וַיֹּ֥אמֶר אֱלֹהִ֖ים יְהִ֣י א֑וֹר וַֽיְהִי־אֽוֹר׃


וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָא֖וֹר כִּי־ט֑וֹב וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָא֖וֹר וּבֵ֥ין הַחֹֽשֶׁךְ׃


וַיִּקְרָ֨א אֱלֹהִ֤ים ׀ לָאוֹר֙ י֔וֹם וְלַחֹ֖שֶׁךְ קָ֣רָא לָ֑יְלָה וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר י֥וֹם אֶחָֽד׃
\p
\aliya{[לוי]}
וַיֹּ֣אמֶר אֱלֹהִ֔ים יְהִ֥י רָקִ֖יעַ בְּת֣וֹךְ הַמָּ֑יִם וִיהִ֣י מַבְדִּ֔יל בֵּ֥ין מַ֖יִם לָמָֽיִם׃


וַיַּ֣עַשׂ אֱלֹהִים֮ אֶת־הָרָקִ֒יעַ֒ וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֙יִם֙ אֲשֶׁר֙ מִתַּ֣חַת לָרָקִ֔יעַ וּבֵ֣ין הַמַּ֔יִם אֲשֶׁ֖ר מֵעַ֣ל לָרָקִ֑יעַ וַֽיְהִי־כֵֽן׃


וַיִּקְרָ֧א אֱלֹהִ֛ים לָֽרָקִ֖יעַ שָׁמָ֑יִם וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר י֥וֹם שֵׁנִֽי׃
\p
\aliya{[ישראל]}
וַיֹּ֣אמֶר אֱלֹהִ֗ים יִקָּו֨וּ הַמַּ֜יִם מִת